In [ ]:
import torch
import sys
import math

In [ ]:
def generate_disc_set(nb):
    input = torch.rand(nb, 2, dtype=torch.float32)
    label = torch.zeros(nb, dtype=torch.int64)
    # input = torch.empty((1000, 2)).normal(mean=0,var)
    dis = 1 / (2 * math.pi)
    for i in range(input.size(0)):
      if ((input[i] - 0.5).square().sum()) < dis:
        # print(i)
        label[i] = 1
      # print(input[i].square().sum())
    # print(label)
    return input, label

In [ ]:
def sigmoid(result):
    for idx,layer in enumerate(result):
        result[idx] = 1 / (1 + math.exp(-layer))   
    return result


In [ ]:
def loss(v, t):
    # print((v - t).pow(2).sum())
    return (v - t).pow(2).sum()


In [ ]:
input, label = generate_disc_set(3)
# input = torch.tensor([[0.5, 0.3], [0.1, 0.9]])
# label = torch.tensor([1, 0])
# test_input, test_label = generate_disc_set(1000)

print('input:', input, ' label:', label)

class Module:
    def __init__(self, layer = 3, neuron = 25):
        self.layer = layer
        self.neu = neuron
        self.lr = 0.01
        self.input_shape = 2
        self.output_shape = 1    

    def init_weight(self):
        self.weight = []
        self.weight.append(torch.rand(self.input_shape, self.neu) * 2 - 1)
        for i in range(1, self.layer):
            self.weight.append(torch.rand(self.neu+1, self.neu) * 2 - 1)
        self.weight.append(torch.rand(self.neu+1, self.output_shape) * 2 - 1)
        # print('weight:', self.weight)
        # print('weight size:', self.weight[1].size())

    def forward_pass(self, train_data):
        result = []
        result.append(torch.mm(train_data.expand(1, -1), self.weight[0]).relu())
        result[0] = torch.cat((result[0], torch.tensor([[1]])), 1)
        for idx in range(1, self.layer):
            result.append(torch.mm(result[idx - 1] ,self.weight[idx]).relu())
            result[idx] = torch.cat((result[idx], torch.tensor([[1]])), 1)
        # result.append(torch.mm(result[self.layer-1], self.weight[self.layer]).relu())
        result.append(sigmoid(torch.mm(result[self.layer-1], self.weight[self.layer])))
        # print(result[-1])
        # result[self.layer] = sigmoid(result[self.layer])
        # print('result:', result)
        return result

    def backward_pass(self, result, target):
        delta = []
        # delta.append(target - result[self.layer])
        # for i in range(result[self.layer].size(1)):
        #     if result[self.layer][0, i] <= 0:
        #       delta[0][i] = 0
        delta_temp = (target - result[self.layer]).mul(result[self.layer].mul(1 - result[self.layer]))
        delta.append(delta_temp)

        
        for k in range(self.layer-1, -1, -1):
            # print('back_lay:', k)
            # print('delta_0:', delta[0])
            # print('weight_size:', self.weight[k + 1].size())
            
            delta_temp = sigma(delta[0], self.weight[k + 1], result[k + 1])
            # print(delta_temp)
            # for i in range(result[k + 1].size(1)):
            #     if result[k + 1][0, i] < 0:
            #       delta_temp[i] = 0
            # print(delta_temp)
            delta.insert(0, delta_temp)
            # print('-----')
        return delta

In [ ]:
acc_loss = 0
x1 = Module(layer=1, neuron=3)
x1.init_weight()

    print('epoch:', r)
    for i in range(len(input)):
        result = x1.forward_pass(input[i])
        print(result)
        acc_loss += loss(label[i], result[-1])
        delta = x1.backward_pass(result, label[i])